# Marketing Churn Analysis
This notebook uses a k-Nearest-Neighbor (kNN) classifier that I wrote from scratch to predict the churn rate for customers.

In [45]:
import pandas as pd
from knn import *
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [49]:
churn_data = pd.read_csv("churn_data.csv")
churn_validation = pd.read_csv("churn_validation.csv")

## Data Exploration

In [6]:
churn_data.info()
print()
churn_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 9 columns):
CustID        128 non-null int64
Gender        128 non-null object
Age           128 non-null int64
Income        128 non-null object
FamilySize    128 non-null int64
Education     128 non-null int64
Calls         128 non-null int64
Visits        128 non-null int64
Churn         128 non-null object
dtypes: int64(6), object(3)
memory usage: 9.1+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 9 columns):
CustID        32 non-null int64
Gender        32 non-null object
Age           32 non-null int64
Income        32 non-null object
FamilySize    32 non-null int64
Education     32 non-null int64
Calls         32 non-null int64
Visits        32 non-null int64
Churn         32 non-null object
dtypes: int64(6), object(3)
memory usage: 2.3+ KB


We will need to perform binary classification

In [7]:
churn_data.Churn.value_counts()

Yes    72
No     56
Name: Churn, dtype: int64

## Data Preparation

In [50]:
def cat_features(dataframe):
    td = pd.DataFrame({'a':[1,2,3], 'b':[1.0, 2.0, 3.0]})
    return [x for x in list(dataframe) if not(dataframe[x].dtype in [td['a'].dtype, td['b'].dtype])]

data_x = churn_data.drop(["Churn"], axis=1)
data_y = churn_data.Churn

valid_x = churn_validation.drop(["Churn"], axis=1)
valid_y = churn_validation.Churn

data_x = pd.get_dummies(data_x, columns=cat_features(data_x))
valid_x = pd.get_dummies(valid_x, columns=cat_features(valid_x))

encoder = preprocessing.LabelEncoder()
data_y = encoder.fit_transform(data_y)
valid_y = encoder.fit_transform(valid_y)

## Train-Test Split

In [47]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.3, random_state = 4)

## Error Metrics

In [9]:
def print_binary_classif_error_report(y_test, preds):
    print('Accuracy: ' + str(accuracy_score(y_test, preds)))
    print('Precison: ' + str(precision_score(y_test, preds)))
    print('Recall: ' + str(recall_score(y_test, preds)))
    print('F1: ' + str(f1_score(y_test, preds)))
    print('ROC AUC: ' + str(roc_auc_score(y_test, preds)))
    print("Confusion Matrix:\n" + str(confusion_matrix(y_test, preds)))

## Compute Best Model


In [10]:
knn = KNN(3, distance.euclidean)
knn.fit(x_train, y_train)
y_hat = knn.predict(x_test)
# print_binary_classif_error_report(y_test, y_hat)

AttributeError: 'DataFrame' object has no attribute 'to_numpy'